# ShopUNow AI Assistant - Agentic RAG Capstone Project

> **Intelligent AI Assistant for Retail Customer & Employee Support**

---

## Project Overview

This notebook demonstrates an **Agentic RAG (Retrieval-Augmented Generation) system** built for ShopUNow, a fictional retail company. The assistant intelligently routes and answers queries from both **internal employees** and **external customers** across multiple departments.

### Key Features

| Feature | Implementation |
|---------|---------------|
| 4+ Departments | HR, IT Support, Billing, Shipping |
| 10-15 QA pairs/dept | 15 per department (60 total) |
| Sentiment Analysis | LLM-based ClassificationPipeline |
| Department Routing | QueryRouter with rules |
| RAG Pipeline | ChromaDB + DynamicKRetriever |
| Human Escalation | Negative/Unknown triggers |
| Stretch Goal | FastAPI REST API deployment |

### Departments

| Department | User Type | Description |
|------------|-----------|-------------|
| **Human Resources** | Internal Employee | PTO, payroll, benefits, policies |
| **IT Support** | Internal Employee | Passwords, VPN, hardware, software |
| **Billing & Payments** | External Customer | Refunds, invoices, payment methods |
| **Shipping & Delivery** | External Customer | Order tracking, returns, delays |

---

**Author:** Mohsin  
**Program:** Analytics Vidhya GenAI Pinnacle Program

## 1. Environment Setup

First, we'll clone the repository and install dependencies.

In [ ]:
# Clone the repository
!git clone https://github.com/crowbarmassage/agentic-rag-assistant.git
%cd agentic-rag-assistant

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt

In [ ]:
# Set up API key (using Colab secrets or manual input)
import os

# Try to get from Colab secrets first
try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("API key loaded from Colab secrets")
except:
    # Fall back to manual input
    from getpass import getpass
    os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')
    print("API key set manually")

## 2. Architecture Overview

The system follows this flow:

```
User Query
    |
    v
+-------------------+
| Classification    |  <- Sentiment + Department detection
+-------------------+
    |
    v
+-------------------+
|     Router        |  <- Decides: RAG or Escalation?
+-------------------+
    |           |
    v           v
+-------+   +------------+
|  RAG  |   | Escalation |
+-------+   +------------+
    |
    v
+-------------------+
| Response + Sources|
+-------------------+
```

### Components

1. **ClassificationPipeline**: Uses LLM to detect sentiment and classify department
2. **QueryRouter**: Rules-based routing (negative sentiment or unknown dept -> escalate)
3. **DynamicKRetriever**: ChromaDB vector search with adaptive result count
4. **ResponseGenerator**: LLM generates answer from retrieved context

## 3. Data Generation

Generate synthetic FAQ data for all 4 departments using LLM.

In [ ]:
# Generate FAQ data (15 QA pairs per department)
!python datagen/generate_faqs_standalone.py --num-pairs 15

In [ ]:
# Verify generated data
import json
from pathlib import Path

data_dir = Path('./data/raw')
total_pairs = 0

print("Generated FAQ Data Summary")
print("=" * 50)

for file in sorted(data_dir.glob('*.json')):
    with open(file) as f:
        data = json.load(f)
    count = len(data.get('qa_pairs', []))
    total_pairs += count
    print(f"{data['department_name']:25} | {count:2} QA pairs | {data['user_type']}")

print("=" * 50)
print(f"Total: {total_pairs} QA pairs across 4 departments")

In [ ]:
# Show sample QA pair from each department
print("\nSample QA Pairs")
print("=" * 70)

for file in sorted(data_dir.glob('*.json')):
    with open(file) as f:
        data = json.load(f)
    qa = data['qa_pairs'][0]
    print(f"\n[{data['department'].upper()}]")
    print(f"Q: {qa['question']}")
    print(f"A: {qa['answer'][:150]}...")

## 4. Vector Store Ingestion

Ingest FAQ data into ChromaDB with embeddings.

In [ ]:
# Ingest data into ChromaDB
from src.vectorstore import ingest_faqs

ingest_faqs(
    data_dir='./data/raw',
    chroma_dir='./data/chroma_db',
    reset_collection=True
)

print("\nData ingestion complete!")

In [ ]:
# Verify ChromaDB contents
from src.vectorstore import ChromaDBClient
from src.providers import EmbeddingProviderFactory
from src.models import Department

embedder = EmbeddingProviderFactory.create('sentence_transformers')
chroma = ChromaDBClient('./data/chroma_db', embedder)

print("ChromaDB Document Counts")
print("=" * 40)
print(f"Total documents: {chroma.get_document_count()}")
print(f"  HR:          {chroma.get_document_count(Department.HR)}")
print(f"  IT Support:  {chroma.get_document_count(Department.IT_SUPPORT)}")
print(f"  Billing:     {chroma.get_document_count(Department.BILLING)}")
print(f"  Shipping:    {chroma.get_document_count(Department.SHIPPING)}")

## 5. Initialize the Orchestrator

The ShopUNowOrchestrator coordinates all pipeline components.

In [ ]:
from src.orchestrator import ShopUNowOrchestrator
from src.config import Settings

# Initialize with settings
settings = Settings(
    llm_provider='openai',
    llm_model='gpt-4o-mini',
    embedding_provider='sentence_transformers',
    embedding_model='all-MiniLM-L6-v2',
    retrieval_min_threshold=0.3,
    retrieval_max_k=10
)

orchestrator = ShopUNowOrchestrator(settings)

print("Orchestrator initialized!")
print(f"  LLM: {orchestrator.llm.provider_name}")
print(f"  Embeddings: {orchestrator.embedding.provider_name} (dim={orchestrator.embedding.dimension})")
print(f"  Documents: {orchestrator.chroma.get_document_count()}")

## 6. Demo: Query Processing

Now let's demonstrate the system with sample queries across all departments.

In [ ]:
from src.models import QueryRequest, QueryResponse, EscalationResponse

def demo_query(query_text: str):
    """Process a query and display the result."""
    print("\n" + "=" * 70)
    print(f"QUERY: {query_text}")
    print("=" * 70)
    
    request = QueryRequest(query=query_text)
    response = orchestrator.process_query(request)
    
    if isinstance(response, EscalationResponse):
        print(f"\n>>> ESCALATED TO HUMAN AGENT <<<")
        print(f"Reason: {response.reason}")
        print(f"Ticket ID: {response.ticket_id}")
        print(f"Message: {response.message}")
    else:
        print(f"\nDepartment: {response.department.value.upper()}")
        print(f"Sentiment: {response.sentiment.value}")
        print(f"Confidence: {response.confidence:.2f}")
        print(f"\nANSWER: {response.answer}")
        print(f"\nSources ({len(response.sources)}):")
        for src in response.sources[:3]:
            print(f"  - [{src.relevance_score:.2f}] {src.question_matched[:60]}...")
        print(f"\nProcessing time: {response.processing_time_ms:.0f}ms")
    
    return response

### 6.1 HR Department (Internal Employee)

In [ ]:
# HR Query - Neutral sentiment -> RAG response
response = demo_query("How do I apply for paid time off?")

### 6.2 IT Support Department (Internal Employee)

In [ ]:
# IT Support Query - Neutral sentiment -> RAG response
response = demo_query("I forgot my password and can't login to my computer")

### 6.3 Billing Department (External Customer)

In [ ]:
# Billing Query - Neutral sentiment -> RAG response
response = demo_query("How can I request a refund for my recent purchase?")

### 6.4 Shipping Department (External Customer)

In [ ]:
# Shipping Query - Neutral sentiment -> RAG response
response = demo_query("Where is my order? I placed it last week.")

### 6.5 Negative Sentiment -> Human Escalation

In [ ]:
# Negative sentiment -> Escalation
response = demo_query("This is absolutely ridiculous! I've been waiting for 3 weeks and nobody can help me!!!")

### 6.6 Unknown Department -> Human Escalation

In [ ]:
# Unknown department -> Escalation
response = demo_query("What's the weather like today?")

### 6.7 Positive Sentiment -> RAG Response

In [ ]:
# Positive sentiment with question -> RAG response (not escalated)
response = demo_query("Thanks so much for your help! Quick question - how do I update my payment method?")

## 7. Component Deep Dive

Let's examine each component in detail.

### 7.1 Classification Pipeline

In [ ]:
from src.pipelines.classification import ClassificationPipeline

classifier = ClassificationPipeline(orchestrator.llm)

# Test sentiment detection
test_queries = [
    "How do I check my PTO balance?",
    "Thank you so much for your help!",
    "This is unacceptable! I demand a refund NOW!"
]

print("Sentiment Analysis Results")
print("=" * 60)

for query in test_queries:
    result = classifier.detect_sentiment(query)
    print(f"\nQuery: {query}")
    print(f"  Sentiment: {result.sentiment.value}")
    print(f"  Confidence: {result.confidence:.2f}")
    print(f"  Indicators: {result.indicators}")

In [ ]:
# Test department classification
test_queries = [
    "I need to apply for vacation days",
    "My laptop won't turn on",
    "I was charged twice for the same order",
    "When will my package arrive?"
]

print("Department Classification Results")
print("=" * 60)

for query in test_queries:
    result = classifier.classify_department(query)
    print(f"\nQuery: {query}")
    print(f"  Department: {result.department.value}")
    print(f"  User Type: {result.user_type.value}")
    print(f"  Confidence: {result.confidence:.2f}")
    print(f"  Reasoning: {result.reasoning}")

### 7.2 Query Router

In [ ]:
from src.routing import QueryRouter
from src.models import ClassificationResult, Sentiment, Department, UserType, RouteDecision

router = QueryRouter()

# Test routing decisions
test_cases = [
    ("Neutral + Valid Dept", Sentiment.NEUTRAL, Department.HR),
    ("Positive + Valid Dept", Sentiment.POSITIVE, Department.BILLING),
    ("Negative + Valid Dept", Sentiment.NEGATIVE, Department.SHIPPING),
    ("Neutral + Unknown Dept", Sentiment.NEUTRAL, Department.UNKNOWN),
]

print("Routing Decision Matrix")
print("=" * 70)
print(f"{'Scenario':25} | {'Sentiment':10} | {'Department':12} | {'Route':20}")
print("-" * 70)

for name, sentiment, dept in test_cases:
    classification = ClassificationResult(
        sentiment=sentiment,
        department=dept,
        user_type=UserType.UNKNOWN,
        confidence=0.9,
        reasoning="test"
    )
    decision = router.route(classification)
    route = "ESCALATE" if decision.should_escalate else "RAG"
    print(f"{name:25} | {sentiment.value:10} | {dept.value:12} | {route:20}")

### 7.3 RAG Retrieval Pipeline

In [ ]:
from src.pipelines.retrieval import DynamicKRetriever

retriever = DynamicKRetriever(
    chroma_client=orchestrator.chroma,
    min_threshold=0.3,
    max_k=10,
    drop_off_ratio=0.7
)

# Test retrieval
result = retriever.retrieve(
    query="How do I request time off for vacation?",
    department=Department.HR
)

print("RAG Retrieval Results")
print("=" * 60)
print(f"Query: {result.query}")
print(f"Department Filter: {result.department_filter.value}")
print(f"Documents Retrieved: {result.retrieval_count}")
print(f"Threshold Used: {result.threshold_used}")
print(f"\nTop Results:")

for doc in result.documents[:5]:
    print(f"\n  [{doc.similarity_score:.3f}] {doc.id}")
    print(f"  Q: {doc.question[:70]}...")

## 8. API Deployment (Stretch Goal)

The system includes a FastAPI REST API for production deployment.

In [ ]:
# Show the API endpoints
print("FastAPI Endpoints")
print("=" * 50)
print("""
POST /query          - Process user query
GET  /health         - Health check
GET  /health?deep=true - Deep health check (all components)
GET  /departments    - List available departments
GET  /docs           - Swagger UI documentation
GET  /redoc          - ReDoc documentation
""")

# Example curl commands
print("\nExample API Usage:")
print("-" * 50)
print("""
# Start the API
python run.py

# Query the assistant
curl -X POST http://localhost:8000/query \\
  -H "Content-Type: application/json" \\
  -d '{"query": "How do I apply for PTO?"}'

# Health check
curl http://localhost:8000/health?deep=true
""")

## 9. Summary

### Requirements Checklist

| Requirement | Status | Implementation |
|-------------|--------|---------------|
| 4+ departments (2 internal, 2 external) | ✅ | HR, IT Support (internal) + Billing, Shipping (external) |
| 10-15 QA pairs per department | ✅ | 15 per department = 60 total |
| Sentiment analysis | ✅ | LLM-based ClassificationPipeline |
| Department routing | ✅ | QueryRouter with rules |
| RAG pipeline | ✅ | ChromaDB + DynamicKRetriever + LLM generation |
| Human escalation | ✅ | Negative sentiment or unknown department |
| 1+ stretch goal | ✅ | FastAPI REST API deployment |

### Architecture Highlights

1. **Provider Abstraction**: Easily swap LLM (OpenAI/Gemini/Groq) and embedding providers
2. **Dynamic K Retrieval**: Adaptive result count based on relevance scores
3. **Metadata Filtering**: Department-specific vector search in ChromaDB
4. **Source Attribution**: Transparent citations for every response
5. **Production Ready**: FastAPI with health checks and CORS support

### Key Files

| File | Purpose |
|------|--------|
| `src/orchestrator.py` | Main pipeline coordinator |
| `src/pipelines/classification.py` | Sentiment + department detection |
| `src/pipelines/retrieval.py` | Dynamic K retrieval logic |
| `src/routing/router.py` | Escalation decision logic |
| `src/vectorstore/chroma_client.py` | ChromaDB wrapper |
| `src/main.py` | FastAPI application |

In [ ]:
print("\n" + "=" * 60)
print("   ShopUNow AI Assistant - Capstone Project Complete!")
print("=" * 60)
print("\nThank you for reviewing this project.")
print("\nFor more details, see:")
print("  - README.md: Project overview and setup")
print("  - TECH_SPECS.md: Detailed technical documentation")
print("  - deliverables/ARCHITECTURE_REPORT.md: Architecture deep-dive")